# Load python packages

In [1]:
import numpy as np
import scanpy as sc
import squidpy as sq
import SpaGCN as spg
import pandas as pd
import seaborn as sns
import scipy
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist
import random, torch
import plotnine as p9

sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=600, facecolor="white")

# Load python functions

## Initial settings: color, order, trait

In [2]:
from initial_settings import *

## Enrich AnnData object with additional gene information

In [3]:
# Parse the 'attribute' column to extract gene names and other desired information
def parse_attributes(attributes_str):
    attributes = {}
    for attribute in attributes_str.split('; '):
        if attribute:
            key, value = attribute.split(' ')
            attributes[key] = value.strip('"')
    return attributes

def enrich_anndata(file_visium, file_gtf, file_out):
    gene_annot = pd.read_csv(file_gtf, sep='\t', comment='#', header=None)
    gene_annot.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']

    # Apply parsing function to the attribute column
    parsed_attri = gene_annot['attribute'].apply(parse_attributes)
    gene_annot = gene_annot.join(parsed_attri.apply(pd.Series))
    gene_annot['gene_id'] = gene_annot['gene_id'].str.split('.').str[0] # strip off the decimal and subsequent version number from Ensembl IDs
    gene_annot['start'] = gene_annot['start'].astype('str')
    gene_annot['end'] = gene_annot['end'].astype('str')
    # gene_annot['gene_name'] = gene_annot['gene_name'].str.split('.').str[0]
    

    # Map Ensembl IDs to official gene names
    gene_id_to_name = dict(zip(gene_annot['gene_id'], gene_annot['gene_name']))
    gene_id_to_type = dict(zip(gene_annot['gene_id'], gene_annot['gene_type']))
    gene_id_to_chr = dict(zip(gene_annot['gene_id'], gene_annot['seqname']))
    gene_id_to_start = dict(zip(gene_annot['gene_id'], gene_annot['start']))
    gene_id_to_end = dict(zip(gene_annot['gene_id'], gene_annot['end']))

    # Update AnnData
    adata = sq.read.visium(file_visium, counts_file='raw_feature_bc_matrix.h5')
    adata.var['gene_name'] = adata.var['gene_ids'].map(gene_id_to_name)
    adata.var['gene_type'] = adata.var['gene_ids'].map(gene_id_to_type)
    adata.var['gene_chrom'] = adata.var['gene_ids'].map(gene_id_to_chr)
    adata.var['gene_start'] = adata.var['gene_ids'].map(gene_id_to_start)
    adata.var['gene_end'] = adata.var['gene_ids'].map(gene_id_to_end)
    adata.write(file_out)

## Identify outliers based on MAD (median absolute deviations)

In [3]:
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

## Quality control

In [4]:
def qc_adata(adata, in_tissue, pct_counts_mt):
    adata.var_names_make_unique()
    adata = adata[adata.obs[in_tissue].astype(bool), :]
    print(adata)
    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)
    
    fig, axs = plt.subplots(1, 4, figsize=(15, 4))
    sns.histplot(adata.obs["total_counts"], kde=False, ax=axs[0])
    sns.histplot(
        adata.obs["total_counts"][adata.obs["total_counts"] < 10000],
        kde=False,
        bins=40,
        ax=axs[1],
    )
    sns.histplot(adata.obs["n_genes_by_counts"], kde=False, bins=60, ax=axs[2])
    sns.histplot(
        adata.obs["n_genes_by_counts"][adata.obs["n_genes_by_counts"] < 4000],
        kde=False,
        bins=60,
        ax=axs[3],
    )
    
    adata = adata[adata.obs["pct_counts_mt"] < pct_counts_mt].copy()
    print(f"#cells after MT filter: {adata.n_obs}")
    return(adata)

## Normalize

In [5]:
def norm_adata(adata):
    sc.pp.normalize_total(adata, inplace=True)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata)
    return(adata)

## Manifold embedding and clustering

In [6]:
def manifold_adata(adata, resolution):
    sc.pp.pca(adata)
    sc.pp.neighbors(adata)
    sc.tl.umap(adata)
    sc.tl.leiden(adata, key_added="clusters", n_iterations=2, resolution = resolution)
    sc.pl.umap(adata, color=["total_counts", "n_genes_by_counts", "clusters"], wspace=0.4)
    sc.pl.spatial(adata, img_key="hires", color=["clusters", "MBP", "MAP2", 'CAMK2A'], size=1.5, cmap='Spectral_r')
    return(adata)

## Obtain spatial domains in Squidpy

In [7]:
def spatial_domain_squidpy(adata, alpha=0.2, resolution=0.3):
    # nearest neighbor graph
    sc.pp.neighbors(adata)
    nn_graph_genes = adata.obsp["connectivities"]
    # spatial proximity graph
    sq.gr.spatial_neighbors(adata)
    nn_graph_space = adata.obsp["spatial_connectivities"]

    # Identidy communities in both representations jointly
    joint_graph = (1 - alpha) * nn_graph_genes + alpha * nn_graph_space
    sc.tl.leiden(adata, adjacency=joint_graph, key_added="squidpy_domains", resolution = resolution)
    sc.pl.spatial(adata, img_key="hires", color=["clusters", "squidpy_domains", "GFAP"], size=1.5, cmap='Spectral_r')
    return(adata)

## Prepare input file for ONTraC

In [ ]:
def prepare_input_for_ontrac(adata, sample, cell_type):
    samples = pd.Series([sample] * adata.n_obs, index=adata.obs_names)
    # cell_types = pd.Series([cell_type] * adata.n_obs, index=adata.obs_names)
    cell_types = adata.obs[cell_type]
    cell_ids = samples + '_' + adata.obs_names
    coordinates = adata.obsm['spatial']
    x_coords = coordinates[:, 0]
    y_coords = coordinates[:, 1]

    # Combine into a DataFrame
    df = pd.DataFrame({
        'Cell_ID': cell_ids,
        'Sample': samples,
        'Cell_Type': cell_types,
        'x': x_coords,
        'y': y_coords
    })

    return(df)

## Examine gene expression difference between clusters

### Plot using sq.pl.spatial_scatter with p-value

#### Store into file

In [3]:
def gene_expr_clusters(adata, gene, cluster_column, labels_c0, labels_c1, file_out, dot_size = 2, dpi = 600):
    # Extract gene expression values
    expr = adata[:, adata.var_names == gene].X.toarray().flatten()
    
    # Extract cluster labels
    clusters = adata.obs[cluster_column]
    
    # Separate based on clusters
    expr_cluster0 = expr[clusters.isin(labels_c0)]
    expr_cluster1 = expr[clusters.isin(labels_c1)]
    
    # Perform a statistical test
    stat, p_val = stats.mannwhitneyu(expr_cluster0, expr_cluster1)
    p_val = "{:.2e}".format(p_val)
    print(f'Statistic: {stat}, P-value: {p_val}')
    # sc.pl.spatial(adata, img_key="hires", color=["clusters", gene], size=1.5, cmap='Spectral_r', dpi=dpi)
    p = sq.pl.spatial_scatter(adata, color=[cluster_column, gene], size=dot_size, cmap='Spectral_r', img=None, dpi=dpi, save = file_out)

#### Plot directly

In [4]:
def gene_expr_clusters_plot_only(adata, gene, cluster_column, labels_c0, labels_c1, dot_size = 2, dpi = 600):
    # Extract gene expression values
    expr = adata[:, adata.var_names == gene].X.toarray().flatten()
    
    # Extract cluster labels
    clusters = adata.obs[cluster_column]
    
    # Separate based on clusters
    expr_cluster0 = expr[clusters.isin(labels_c0)]
    expr_cluster1 = expr[clusters.isin(labels_c1)]
    
    # Perform a statistical test
    stat, p_val = stats.mannwhitneyu(expr_cluster0, expr_cluster1)
    p_val = "{:.2e}".format(p_val)
    print(f'Statistic: {stat}, P-value: {p_val}')
    # sc.pl.spatial(adata, img_key="hires", color=["clusters", gene], size=1.5, cmap='Spectral_r', dpi=dpi)
    p = sq.pl.spatial_scatter(adata, color=[cluster_column, gene], size=dot_size, cmap='Spectral_r', img=None, dpi=dpi)

### Plot in boxplot format

In [13]:
def gene_expr_clusters_in_boxplot(adata, gene, cluster_column, labels_c0, labels_c1, file_out, figsize=(4, 6), dpi=600, ylim=None):
    # Extract gene expression values
    expr = adata[:, adata.var_names == gene].X.toarray().flatten()
    
    # Extract cluster labels
    clusters = adata.obs[cluster_column]
    
    # Separate based on clusters
    expr_cluster0 = expr[clusters.isin(labels_c0)]
    expr_cluster1 = expr[clusters.isin(labels_c1)]
    
    # Perform a statistical test
    stat, p_val = stats.mannwhitneyu(expr_cluster0, expr_cluster1)
    p_val_text = "p = {:.2e}".format(p_val)  # Format p-value for display
    
    # Plotting
    fig, ax = plt.subplots(figsize=figsize)
    sns.boxplot(data=[expr_cluster0, expr_cluster1], ax=ax, palette=['#F4CD9E', '#9A7B56'])
    ax.set_xticklabels(['Cluster 0', 'Cluster 1'])
    ax.set_ylabel('Expression')
    ax.set_title(f'Expression of {gene}')

    # Set custom y-axis limits if provided
    if ylim:
        ax.set_ylim(ylim)

    # Calculate the maximum y-value from both clusters to determine line position for p-value annotation
    max_y = max(np.max(expr_cluster0), np.max(expr_cluster1))

    # Add p-value annotation
    x1, x2 = 0, 1  # columns 'Cluster 0' and 'Cluster 1' on x-axis
    y = max_y + np.abs(max_y * 0.05)  # 5% higher than the max
    h, col = np.abs(max_y * 0.03), 'k'  # height of the line is 3% of the max
    ax.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
    ax.text((x1+x2)*.5, y+h, p_val_text, ha='center', va='bottom', color=col)

    # Save the plot
    plt.savefig(file_out, dpi=dpi)
    plt.show()



# Basic analysis from filtered feature (pct_counts_mt < 30)

## Load data

In [ ]:
for sub_sample in ['A', 'B', 'C', 'D']:
    enrich_anndata(file_visium='sample_' + sub_sample + '_loupe/outs', 
                   file_gtf='GRCh38_basic_pri.gtf', 
                   file_out='sample_' + sub_sample + '_add_geneInfo.h5ad')

In [ ]:
adata_A = sc.read('sample_A_add_geneInfo.h5ad')
adata_B = sc.read('sample_B_add_geneInfo.h5ad')
adata_C = sc.read('sample_C_add_geneInfo.h5ad')
adata_D = sc.read('sample_D_add_geneInfo.h5ad')

## Quality control

In [ ]:
adata_A = qc_adata(adata_A, 'in_tissue', 30)
adata_B = qc_adata(adata_B, 'in_tissue', 30)
adata_C = qc_adata(adata_C, 'in_tissue', 30)
adata_D = qc_adata(adata_D, 'in_tissue', 30)

## Only focus on protein-coding genes on eurochromatin

In [ ]:
adata_A_pc_euch = adata_A[:, (adata_A.var['gene_type'] == 'protein_coding') & (adata_A.var['gene_chrom'].isin([f'chr{i}' for i in range(1, 23)]))].copy()
adata_B_pc_euch = adata_B[:, (adata_B.var['gene_type'] == 'protein_coding') & (adata_B.var['gene_chrom'].isin([f'chr{i}' for i in range(1, 23)]))].copy()
adata_C_pc_euch = adata_C[:, (adata_C.var['gene_type'] == 'protein_coding') & (adata_C.var['gene_chrom'].isin([f'chr{i}' for i in range(1, 23)]))].copy()
adata_D_pc_euch = adata_D[:, (adata_D.var['gene_type'] == 'protein_coding') & (adata_D.var['gene_chrom'].isin([f'chr{i}' for i in range(1, 23)]))].copy()

## Normalize

In [ ]:
adata_A_pc_euch = norm_adata(adata_A_pc_euch)
adata_B_pc_euch = norm_adata(adata_B_pc_euch)
adata_C_pc_euch = norm_adata(adata_C_pc_euch)
adata_D_pc_euch = norm_adata(adata_D_pc_euch)

## Manifold embed and clust

In [ ]:
adata_A_pc_euch = manifold_adata(adata_A_pc_euch, resolution=0.3)
adata_B_pc_euch = manifold_adata(adata_B_pc_euch, resolution=0.3)
adata_C_pc_euch = manifold_adata(adata_C_pc_euch, resolution=0.3)
adata_D_pc_euch = manifold_adata(adata_D_pc_euch, resolution=0.3)

## Obtain spatial domains in Squidpy

In [ ]:
adata_A_pc_euch = spatial_domain_squidpy(adata_A_pc_euch)
adata_B_pc_euch = spatial_domain_squidpy(adata_B_pc_euch)
adata_C_pc_euch = spatial_domain_squidpy(adata_C_pc_euch)
adata_D_pc_euch = spatial_domain_squidpy(adata_D_pc_euch)

In [24]:
adata_A_pc_euch.write('sample_A_pc_euch_clustered_pct_mt_30.h5ad')
adata_B_pc_euch.write('sample_B_pc_euch_clustered_pct_mt_30.h5ad')
adata_C_pc_euch.write('sample_C_pc_euch_clustered_pct_mt_30.h5ad')
adata_D_pc_euch.write('sample_D_pc_euch_clustered_pct_mt_30.h5ad')

## Expression of marker genes

### MBP

In [ ]:
# sample_a
sq.pl.spatial_scatter(
    adata_A_pc_euch, 
    color=['MBP'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_a_cluster_mbp.pdf', 
    dpi=300,
    cmap=cmap_plot  # Specify the color map directly
)

# sample_b
sq.pl.spatial_scatter(
    adata_B_pc_euch, 
    color=['MBP'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_b_cluster_mbp.pdf', 
    dpi=300,
    cmap=cmap_plot  # Specify the color map directly
)

# sample_c
sq.pl.spatial_scatter(
    adata_C_pc_euch, 
    color=['MBP'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_c_cluster_mbp.pdf', 
    dpi=300,
    cmap=cmap_plot  # Specify the color map directly
)

# sample_d
sq.pl.spatial_scatter(
    adata_D_pc_euch, 
    color=['MBP'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_d_cluster_mbp.pdf', 
    dpi=300,
    cmap=cmap_plot  # Specify the color map directly
)

### GFAP

In [ ]:
# sample_a
sq.pl.spatial_scatter(
    adata_A_pc_euch, 
    color=['GFAP'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_a_cluster_gfap.pdf', 
    dpi=300,
    cmap=cmap_plot  # Specify the color map directly
)

# sample_b
sq.pl.spatial_scatter(
    adata_B_pc_euch, 
    color=['GFAP'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_b_cluster_gfap.pdf', 
    dpi=300,
    cmap=cmap_plot  # Specify the color map directly
)

# sample_c
sq.pl.spatial_scatter(
    adata_C_pc_euch, 
    color=['GFAP'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_c_cluster_gfap.pdf', 
    dpi=300,
    cmap=cmap_plot  # Specify the color map directly
)

# sample_d
sq.pl.spatial_scatter(
    adata_D_pc_euch, 
    color=['GFAP'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_d_cluster_gfap.pdf', 
    dpi=300,
    cmap=cmap_plot  # Specify the color map directly
)

### SLC1A2

In [ ]:
# sample_a
sq.pl.spatial_scatter(
    adata_A_pc_euch, 
    color=['SLC1A2'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_a_cluster_slc1a2.pdf', 
    dpi=300,
    cmap=cmap_plot  # Specify the color map directly
)

# sample_b
sq.pl.spatial_scatter(
    adata_B_pc_euch, 
    color=['SLC1A2'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_b_cluster_slc1a2.pdf', 
    dpi=300,
    cmap=cmap_plot  # Specify the color map directly
)

# sample_c
sq.pl.spatial_scatter(
    adata_C_pc_euch, 
    color=['SLC1A2'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_c_cluster_slc1a2.pdf', 
    dpi=300,
    cmap=cmap_plot  # Specify the color map directly
)

# sample_d
sq.pl.spatial_scatter(
    adata_D_pc_euch, 
    color=['SLC1A2'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_d_cluster_slc1a2.pdf', 
    dpi=300,
    cmap=cmap_plot  # Specify the color map directly
)

## Identify Spatial domains using ONTraC

### Prepare the input files

In [ ]:
df_A = prepare_input_for_ontrac(adata_A_pc_euch, 'A', 'squidpy_domains') 
df_B = prepare_input_for_ontrac(adata_B_pc_euch, 'B', 'squidpy_domains') 
df_C = prepare_input_for_ontrac(adata_C_pc_euch, 'C', 'squidpy_domains') 
df_D = prepare_input_for_ontrac(adata_D_pc_euch, 'D', 'squidpy_domains')
combined_df = pd.concat([df_A, df_B, df_C, df_D], ignore_index=True)

In [ ]:
combined_df.to_csv('original_data.csv', index=False)

### Run ONTraC

In [ ]:
%%sh

cd ${dataPATH}
ONTraC -d ${dataPATH}/original_data.csv --preprocessing-dir ${dataPATH}/preprocessing --GNN-dir ${dataPATH}/GNN --NTScore-dir ${dataPATH}/NTScore

### Spatial domains visualization

In [ ]:
domains_ontrac = pd.read_csv('ontrac/GNN/cell_level_max_niche_cluster.csv', index_col=0)
# sample_A
mask_A = domains_ontrac.index.str.startswith('A_')
subset_df_A = domains_ontrac[mask_A]
subset_df_A.index = subset_df_A.index.str.replace('A_', '')
adata_A_pc_euch.obs['Niche_Cluster'] = subset_df_A['Niche_Cluster'].astype(str)

# sample_B
mask_B = domains_ontrac.index.str.startswith('B_')
subset_df_B = domains_ontrac[mask_B]
subset_df_B.index = subset_df_B.index.str.replace('B_', '')
adata_B_pc_euch.obs['Niche_Cluster'] = subset_df_B['Niche_Cluster'].astype(str)

# sample_C
mask_C = domains_ontrac.index.str.startswith('C_')
subset_df_C = domains_ontrac[mask_C]
subset_df_C.index = subset_df_C.index.str.replace('C_', '')
adata_C_pc_euch.obs['Niche_Cluster'] = subset_df_C['Niche_Cluster'].astype(str)

# sample_D
mask_D = domains_ontrac.index.str.startswith('D_')
subset_df_D = domains_ontrac[mask_D]
subset_df_D.index = subset_df_D.index.str.replace('D_', '')
adata_D_pc_euch.obs['Niche_Cluster'] = subset_df_D['Niche_Cluster'].astype(str)

In [ ]:
# sample_a
sq.pl.spatial_scatter(
    adata_A_pc_euch, 
    color=['Niche_Cluster'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_a_cluster_niche_cluster.pdf', 
    dpi=300,
    palette=cmap_niche  # Specify the color map directly
)

# sample_b
sq.pl.spatial_scatter(
    adata_B_pc_euch, 
    color=['Niche_Cluster'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_b_cluster_niche_cluster.pdf', 
    dpi=300,
    palette=cmap_niche  # Specify the color map directly
)

# sample_c
sq.pl.spatial_scatter(
    adata_C_pc_euch, 
    color=['Niche_Cluster'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_c_cluster_niche_cluster.pdf', 
    dpi=300,
    palette=cmap_niche  # Specify the color map directly
)

# sample_d
sq.pl.spatial_scatter(
    adata_D_pc_euch, 
    color=['Niche_Cluster'], 
    size=1.5, 
    legend_fontsize=10, 
    legend_fontoutline=2, 
    outline_width=(0.5, 0.3), 
    frameon=False,
    img=None, 
    save='ast_visium_sample_d_cluster_niche_cluster.pdf', 
    dpi=300,
    palette=cmap_niche  # Specify the color map directly
)

### Examine gene expression between spatial domains

#### MBP

In [ ]:
gene_expr_clusters_in_boxplot(adata_A_pc_euch, 'MBP', "Niche_Cluster", ['0'], ['1'], file_out='ast_visium_sample_a_cluster_mbp_boxplot.pdf', figsize=(3.5, 6), ylim=(0, 6))
gene_expr_clusters_in_boxplot(adata_B_pc_euch, 'MBP', "Niche_Cluster", ['0'], ['1'], file_out='ast_visium_sample_b_cluster_mbp_boxplot.pdf', figsize=(3.5, 6), ylim=(0, 6))
gene_expr_clusters_in_boxplot(adata_C_pc_euch, 'MBP', "Niche_Cluster", ['0'], ['1'], file_out='ast_visium_sample_c_cluster_mbp_boxplot.pdf', figsize=(3.5, 6), ylim=(0, 6))
gene_expr_clusters_in_boxplot(adata_D_pc_euch, 'MBP', "Niche_Cluster", ['0'], ['1'], file_out='ast_visium_sample_d_cluster_mbp_boxplot.pdf', figsize=(3.5, 6), ylim=(0, 6))

#### GFAP

In [ ]:
gene_expr_clusters_in_boxplot(adata_A_pc_euch, 'GFAP', "Niche_Cluster", ['0'], ['1'], file_out='ast_visium_sample_a_cluster_gfap_boxplot.pdf', figsize=(3.5, 6), ylim=(0, 6))
gene_expr_clusters_in_boxplot(adata_B_pc_euch, 'GFAP', "Niche_Cluster", ['0'], ['1'], file_out='ast_visium_sample_b_cluster_gfap_boxplot.pdf', figsize=(3.5, 6), ylim=(0, 6))
gene_expr_clusters_in_boxplot(adata_C_pc_euch, 'GFAP', "Niche_Cluster", ['0'], ['1'], file_out='ast_visium_sample_c_cluster_gfap_boxplot.pdf', figsize=(3.5, 6), ylim=(0, 6))
gene_expr_clusters_in_boxplot(adata_D_pc_euch, 'GFAP', "Niche_Cluster", ['0'], ['1'], file_out='ast_visium_sample_d_cluster_gfap_boxplot.pdf', figsize=(3.5, 6), ylim=(0, 6))

#### SLC1A2

In [ ]:
gene_expr_clusters_in_boxplot(adata_A_pc_euch, 'SLC1A2', "Niche_Cluster", ['0'], ['1'], file_out='ast_visium_sample_a_cluster_slc1a2_boxplot.pdf', figsize=(3.5, 6), ylim=(0, 4))
gene_expr_clusters_in_boxplot(adata_B_pc_euch, 'SLC1A2', "Niche_Cluster", ['0'], ['1'], file_out='ast_visium_sample_b_cluster_slc1a2_boxplot.pdf', figsize=(3.5, 6), ylim=(0, 4))
gene_expr_clusters_in_boxplot(adata_C_pc_euch, 'SLC1A2', "Niche_Cluster", ['0'], ['1'], file_out='ast_visium_sample_c_cluster_slc1a2_boxplot.pdf', figsize=(3.5, 6), ylim=(0, 4))
gene_expr_clusters_in_boxplot(adata_D_pc_euch, 'SLC1A2', "Niche_Cluster", ['0'], ['1'], file_out='ast_visium_sample_d_cluster_slc1a2_boxplot.pdf', figsize=(3.5, 6), ylim=(0, 4))